In [5]:
#Configuring paht of kaggle.json file
import os
import shutil

# Create .kaggle directory in the user home folder
kaggle_dir = os.path.join(os.path.expanduser("~"), ".kaggle")
os.makedirs(kaggle_dir, exist_ok=True)

# Copy kaggle.json to the .kaggle directory
shutil.copy("kaggle.json", kaggle_dir)

# Set permissions to the file (if needed)
os.chmod(os.path.join(kaggle_dir, "kaggle.json"), 0o600)

### Importing Twitter Sentiment Analysis Dataset

In [2]:
# Fetching data thorugh API
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
from zipfile import ZipFile

dataset = 'sentiment140.zip'

with ZipFile(dataset,'r') as zip:
    zip.extractall()
    print('The dataset is extracted')

The dataset is extracted


### Importing Libraries

In [4]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mhtsa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
#Printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Data Pre-Processing

In [7]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [8]:
df.head(3)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire


In [9]:
df.columns

Index(['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY',
       '_TheSpecialOne_',
       '@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D'],
      dtype='object')

In [10]:
df.shape

(1599999, 6)

In [11]:
column_names = ['target','id','date','flag','user','text']
df = pd.read_csv('training.1600000.processed.noemoticon.csv',names=column_names,encoding='ISO-8859-1')
df.head(3)

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...


In [12]:
df.shape

(1600000, 6)

In [13]:
df.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [14]:
df['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [15]:
df.replace({'target':{4:1}}, inplace=True)

In [16]:
df['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 - Negative Tweet,
1 - Positive Tweet

In [17]:
port_stem = PorterStemmer()

In [18]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-z]', ' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [19]:
df['stemmed_content'] = df['text'].apply(stemming)

In [20]:
df.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [21]:
print(df['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [39]:
# Separating data and label
x = df['stemmed_content'].values
y = df['target'].values

In [40]:
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [41]:
print(y)

[0 0 0 ... 1 1 1]


In [42]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [43]:
print(x_train.shape,x_test.shape,x.shape)

(1280000,) (320000,) (1600000,)


In [44]:
print(x_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [45]:
print(x_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [46]:
# Converting text to vectors
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [47]:
print(x_train)

  (0, 464940)	0.4483581441053542
  (0, 247667)	0.4215060595161854
  (0, 116318)	0.3749338694119814
  (0, 194823)	0.5274064910069903
  (0, 372989)	0.3583819096048567
  (0, 458421)	0.27228112326228093
  (1, 169453)	1.0
  (2, 304223)	0.1678648733603879
  (2, 140239)	0.20289994039686138
  (2, 159240)	0.18800868496584808
  (2, 187474)	0.16194060015844067
  (2, 429906)	0.15168139293444055
  (2, 281186)	0.24137202118772233
  (2, 465311)	0.33480686186877023
  (2, 83506)	0.31303964332256906
  (2, 455221)	0.3296073093697297
  (2, 427034)	0.32103058232997417
  (2, 137276)	0.29069581876534506
  (2, 427967)	0.18706371586911208
  (2, 132152)	0.1891875792321668
  (2, 116318)	0.45905705320412793
  (3, 181609)	0.3745702863460754
  (3, 432396)	0.2708400841098632
  (3, 408311)	0.3941744062368762
  (3, 61032)	0.52009780709937
  :	:
  (1279996, 409874)	0.21996814268672968
  (1279996, 455685)	0.27108924963905606
  (1279996, 335154)	0.21190629789318696
  (1279996, 250767)	0.22319402802219876
  (1279996, 3069

In [48]:
print(x_test)

  (0, 442233)	0.17903569516819148
  (0, 429906)	0.3140986667345934
  (0, 419078)	0.34884649404705365
  (0, 408024)	0.21977439030363208
  (0, 294440)	0.17813419553395948
  (0, 285841)	0.4532311769636366
  (0, 180544)	0.2803743470609849
  (0, 146252)	0.23682534421364126
  (0, 140294)	0.25509664355434203
  (0, 112919)	0.367081693610678
  (0, 72943)	0.2678484639307855
  (0, 34872)	0.16236726920078096
  (0, 17977)	0.17180827032540266
  (1, 385085)	0.24302725359248564
  (1, 366102)	0.4820488545932507
  (1, 270721)	0.28432607616832933
  (1, 229396)	0.40557814327518943
  (1, 153709)	0.5722666578553444
  (1, 17977)	0.2084958588978494
  (1, 8801)	0.30376315944222493
  (2, 420861)	0.4330300886388968
  (2, 270795)	0.2562344190631401
  (2, 192882)	0.5885804190746107
  (2, 95488)	0.363168936008175
  (2, 38175)	0.37941511785250515
  :	:
  (319994, 130897)	0.4539824479197656
  (319995, 466849)	0.3209500415545892
  (319995, 442233)	0.22620821621683035
  (319995, 437352)	0.23804373106127552
  (319995, 3

### Training the Logistic Regression Model

In [49]:
model = LogisticRegression(max_iter=1000)

In [50]:
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

In [51]:
y_train_pred = model.predict(x_train)
train_acc = accuracy_score(y_train,y_train_pred)
print(f"Training Accuracy is: {train_acc}")

Training Accuracy is: 0.79812734375


In [54]:
y_test_pred = model.predict(x_test)
test_acc = accuracy_score(y_test,y_test_pred)
print(f"Testing Accuracy is: {test_acc}")
print(f"Model Accuracy is {test_acc*100}%")

Testing Accuracy is: 0.777265625
Model Accuracy is 77.7265625%


### Saving the Trained Model

In [55]:
import pickle

In [56]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [57]:
# Using model for future Predictions 
loaded_model = pickle.load(open('trained_model.sav','rb'))

In [58]:
x_new = x_test[200]
print(y_test[200])

1


In [59]:
prediction = model.predict(x_new)
print(prediction)

if prediction==0:
    print("Negative Tweet")
else:
    print("Positive Tweet")

[1]
Positive Tweet
